In [1]:
!pip install transformers datasets

import pandas as pd
from datasets import Dataset
from transformers import BertTokenizer, BertForMultipleChoice, Trainer, TrainingArguments, DataCollatorForMultipleChoice
from sklearn.model_selection import train_test_split
from transformers import DataCollatorForMultipleChoice


In [2]:
import pandas as pd
from datasets import Dataset

# Muat dataset
df = pd.read_csv("toefl_writting.csv", encoding="ISO-8859-1", on_bad_lines='skip')

# Bersihkan duplikasi dan pastikan semua jawaban valid
df.drop_duplicates(subset="question", keep="first", inplace=True)
df = df[df["answer"].isin(["A", "B", "C", "D"])]

# Encode jawaban: A=0, B=1, dst.
df["label"] = df["answer"].apply(lambda x: ord(x) - 65)  # A=0, B=1, C=2, D=3

# Konversi ke Dataset HuggingFace
raw_dataset = Dataset.from_pandas(df)
df.head()

,type,question,A,B,C,D,answer,label
0,Structure,________ raining when we left the building.,It was,There was,It is,There is,A,0
1,Structure,Not only he but also his friends ______ late.,is,are,was,be,B,1
2,Structure,Each of the students ______ a schedule.,has,have,having,to have,A,0
3,Structure,The teacher made the students ______ quiet.,be,to be,being,been,A,0
4,Structure,She said that she ______ to the party.,will go,go,would go,goes,C,2


In [3]:
# Jumlah baris duplikat (seluruh kolom sama persis)
duplikat = df.duplicated().sum()
print(f"Jumlah baris duplikat: {duplikat}")


Jumlah baris duplikat: 0


In [4]:
df.drop_duplicates(subset=["question"], inplace=True)


In [5]:
from transformers import BertTokenizer
from datasets import Dataset

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
num_choices = 4

def preprocess_function(examples):
    # Gabungkan soal dan pilihan jawaban
    questions = [[q] * num_choices for q in examples["question"]]  # Ulangi pertanyaan 4x per soal

    # Gabungkan pilihan jawaban sebagai list of lists
    choices_list = [list(tup) for tup in zip(examples["A"], examples["B"], examples["C"], examples["D"])]

    # Flatten untuk tokenisasi
    flat_questions = sum(questions, [])
    flat_choices = sum(choices_list, [])

    # Tokenisasi
    tokenized = tokenizer(
        flat_questions,
        flat_choices,
        truncation=True,
        padding="max_length",
        max_length=128,
        return_tensors="pt"
    )

    # Sesuaikan dimensi agar cocok dengan model BERT multiple choice
    return {
        k: [v[i : i + num_choices].tolist() for i in range(0, len(v), num_choices)]
        for k, v in tokenized.items()
    }

# Pastikan df sudah dibersihkan dan memiliki kolom: question, A, B, C, D, answer
raw_dataset = Dataset.from_pandas(df)

# Jalankan preprocessing
tokenized_dataset = raw_dataset.map(preprocess_function, batched=True, batch_size=16)

def reformat_dataset(tokenized_dataset):
    def _reformat(examples):
        return {
            "input_ids": examples["input_ids"],
            "attention_mask": examples["attention_mask"],
            "token_type_ids": examples["token_type_ids"],
            "labels": examples["label"]
        }

    return tokenized_dataset.map(
        _reformat,
        batched=True,
        remove_columns=tokenized_dataset.column_names
    )

encoded_dataset = reformat_dataset(tokenized_dataset).train_test_split(test_size=0.2)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Map:   0%|          | 0/177 [00:00<?, ? examples/s]

Map:   0%|          | 0/177 [00:00<?, ? examples/s]

In [6]:

# Setup model
model = BertForMultipleChoice.from_pretrained("bert-base-uncased")

training_args = TrainingArguments(
    output_dir="./bert-mc-output",
    num_train_epochs=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    logging_dir="./logs",
    logging_steps=20,
    weight_decay=0.01,
    save_total_limit=2,
    # jangan pakai evaluation_strategy
)


# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer,
)

# Start training
trainer.train()

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-6-7d87a51f42b0>:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: reswan (reswan-universitas-sebelas-april) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
20,1.314900
40,1.115900
60,0.938800
80,0.795100


TrainOutput(global_step=90, training_loss=1.007555490069919, metrics={'train_runtime': 204.9385, 'train_samples_per_second': 6.88, 'train_steps_per_second': 0.439, 'total_flos': 370983257118720.0, 'train_loss': 1.007555490069919, 'epoch': 10.0})

In [7]:
results = trainer.evaluate()
print(results)


{'eval_loss': 1.5197570323944092, 'eval_runtime': 1.1386, 'eval_samples_per_second': 31.617, 'eval_steps_per_second': 2.635, 'epoch': 10.0}


In [8]:
trainer.save_model("./my-mc-model")
tokenizer.save_pretrained("./my-mc-model")


('./my-mc-model/tokenizer_config.json',
 './my-mc-model/special_tokens_map.json',
 './my-mc-model/vocab.txt',
 './my-mc-model/added_tokens.json')

In [9]:
import torch

# Pastikan model ke device (cuda jika tersedia)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()  # Set model ke evaluasi

# Data pertanyaan
new_question = {
    "question": "The news ______ shocking to everyone",
    "A": "was",
    "B": "is",
    "C": "are",
    "D": "has"
}
# Jawaban: A

# Preprocess input
choices = [new_question["A"], new_question["B"], new_question["C"], new_question["D"]]
question_repeat = [new_question["question"]] * len(choices)

# Tokenisasi
encoded = tokenizer(
    question_repeat,
    choices,
    truncation=True,
    padding="max_length",
    max_length=128,
    return_tensors="pt"
)

# Format ulang input agar shape: (1, num_choices, seq_len), lalu pindahkan ke device yang sama
input_dict = {
    k: v.unsqueeze(0).to(device)  # Tambah batch dimensi dan pindah ke device
    for k, v in encoded.items()
}

# Predict
with torch.no_grad():
    outputs = model(**input_dict)
    predicted_label = outputs.logits.argmax(dim=1).item()
    predicted_answer = chr(65 + predicted_label)

print("Jawaban model:", predicted_answer)


Jawaban model: B
